In [1]:
from typing import List
import uproot
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker
import os

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [2]:
MAIN_DIR = "../../"

plot_dir = f"{MAIN_DIR}/plots/GoF/23Oct9"
_ = os.system(f"mkdir -p {plot_dir}")

In [24]:
# cards_dir = "23May13MP"

file = uproot.concatenate(
    f"/ospool/cms-user/yuzhe/BoostedHWW/prediction/boostedHWW/combine/scripts/f_test/cards/nTF_3/higgsCombineData.GoodnessOfFit.mH125.root"
)

data_ts = file["limit"][0]  # data test statistic

file = uproot.concatenate(
    f"/ospool/cms-user/yuzhe/BoostedHWW/prediction/boostedHWW/combine/scripts/f_test/cards/nTF_3/higgsCombineTest.GoodnessOfFit.mH125.root"
)
# //
toy_ts = np.array(file["limit"])  # toy test statistics

ValueError: no TTrees found
in file /ospool/cms-user/yuzhe/BoostedHWW/prediction/boostedHWW/combine/scripts/f_test/cards/nTF_3/higgsCombineTest.GoodnessOfFit.mH125.root

In [22]:
data_ts 

5642.3164473846555

In [23]:
# toy_ts = np.array(file["limit"])
toy_ts

array([], dtype=float64)

In [11]:
def p_value(data_ts: float, toy_ts: List[float]):
    return np.mean(toy_ts >= data_ts)

In [12]:
pval = p_value(data_ts, toy_ts)
pval

/home/yuzhe/miniconda3/envs/bbVV/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/yuzhe/miniconda3/envs/bbVV/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

In [13]:
plot_max = max(np.max(toy_ts), data_ts)
plot_min = min(np.min(toy_ts), data_ts)

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
plt.figure(figsize=(12, 8))
_ = plt.hist(
    toy_ts, np.linspace(plot_min, plot_max, 10), color="dimgrey", histtype="step", label="Toys"
)
plt.axvline(data_ts, color="red", linestyle=":", label=rf"Data ($p$-value = {pval:.2f})")
_ = plt.legend()
plt.title("Goodness of Fit")
plt.ylabel("Number of Toys")
plt.xlabel("Test Statistics")
plt.savefig(f"{plot_dir}/gof.pdf")

In [ ]:
from scipy.stats import chi2

chi2_df = 100

plt.figure(figsize=(12, 8))
x = np.linspace(plot_min, plot_max, 101)
plt.plot(
    x,
    chi2.pdf(x, chi2_df) * 1.3,
    color="blue",
    linestyle="--",
    alpha=0.6,
    label=rf"$\chi^2_{{DoF = {chi2_df}}}$",
)
_ = plt.hist(
    toy_ts,
    np.linspace(plot_min, plot_max, 10),
    density=True,
    color="dimgrey",
    histtype="step",
    label="Toys",
)
plt.axvline(data_ts, color="red", linestyle=":", label=rf"Data ($p$-value = {pval:.2f})")
_ = plt.legend()
plt.title("Goodness of Fit")
plt.ylabel("Number of Toys (A.U.)")
plt.xlabel("Test Statistics")
plt.savefig(f"{plot_dir}/gof_chi2.pdf")